# Feature Exploration

In this section, we are going to compute some of the possible features we 
will use for training the NN. Since the other projects I am working on
do not have all of the data used in the WESAD set, we are only using a subset
of the raw data to compute our features here. Additionally, we are going to use only the chest sensor measurements. This will include the following three sensor values:
1. ACC -> Accelerometer data
2. EDA -> Electrodermal Activity aka GSR (Galvanic Skin Response)
3. TEMP -> Skin Temperature

(Side note: Technically, my other project does have HRV but since it isn't currently being calculated, we won't use it here.)

# Experiments

First we will try out the Data module we created to encapsulate importing and extracting the baseline and stress data for a given subject.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import importlib
import os
import sys
# module = os.path.abspath('/home/learner/DLA_project/src/main')
module = os.path.abspath("C:/Users\\18145\\development\\wesad_experiments\\src\\main")
if module not in sys.path:
    sys.path.append(module)
from Data import Data

In [2]:
data = Data()
subject_three = data.load(3) # load subject two data in two dicts, baseline and stress

Loading data from S3
Path=C:\WESAD\S3\S3.pkl
{'signal': {'chest': {'ACC': array([[ 7.20000029e-01, -1.38199985e-01,  3.19339991e+00],
       [ 5.68799973e-01, -2.28600025e-01,  3.18219995e+00],
       [ 3.99799943e-01, -3.13199997e-01,  2.94499993e+00],
       ...,
       [ 9.01800036e-01,  5.85999489e-02, -7.79998302e-03],
       [ 9.07400012e-01,  7.62000084e-02,  2.59995461e-03],
       [ 9.68199968e-01,  1.83400035e-01,  1.79999948e-01]]), 'ECG': array([[-0.75663757],
       [-0.70820618],
       [-0.67396545],
       ...,
       [-0.24847412],
       [-0.18598938],
       [-0.14176941]]), 'EMG': array([[ 0.00595093],
       [-0.0385437 ],
       [-0.04943848],
       ...,
       [-0.11604309],
       [-0.11082458],
       [-0.06486511]]), 'EDA': array([[7.98835754],
       [7.98225403],
       [7.98339844],
       ...,
       [7.53211975],
       [7.53059387],
       [7.5340271 ]]), 'Temp': array([[31.499023],
       [31.501953],
       [31.521149],
       ...,
       [33.228973],

In [3]:
# Let us take a quick look and make sure the Data module is doing what we want.
print('baseline: ', subject_three[0], '\nstress: ', subject_three[1])

baseline:  {'ACC': array([[ 0.78659999, -0.05980003,  0.50479996],
       [ 0.79100001, -0.06160003,  0.50479996],
       [ 0.79299998, -0.06260002,  0.50179994],
       ...,
       [ 0.85220003, -0.0194    , -0.3592    ],
       [ 0.85099995, -0.01980001, -0.3574    ],
       [ 0.85099995, -0.01899999, -0.35619998]]), 'EDA': array([[7.2227478 ],
       [7.22427368],
       [7.2227478 ],
       ...,
       [6.61582947],
       [6.62078857],
       [6.61735535]]), 'Temp': array([[32.989807],
       [33.030365],
       [33.04239 ],
       ...,
       [31.658417],
       [31.667297],
       [31.662842]], dtype=float32)} 
stress:  {'ACC': array([[ 0.8872    ,  0.06700003,  0.074     ],
       [ 0.88820004,  0.06900001,  0.06579995],
       [ 0.89139998,  0.06819999,  0.06159997],
       ...,
       [ 0.91659999,  0.02499998, -0.0248    ],
       [ 0.91540003,  0.02520001, -0.01980001],
       [ 0.91499996,  0.02359998, -0.01660001]]), 'EDA': array([[6.88705444],
       [6.88667297],
      

### Comments
The output above looks good. We will use a sliding window for feature calculations. I think that the window for ACC will certainly need to be different than the window for EDA/Temp. 

For the chest device, they had a sampling rate of 700 and a resolution of 16. 

### Features

The eatures of interest for the ACC data are as follows.

1. We are interested in computing mean and STD for each axis; I believe that some of the papers, in addition to the WESAD paper, recommend summing those values over all the axes - not sure if we want to do that or not.
2. Peak frequency for each axis
3. RMS = sqrt(x^2 + y^2 + z^2) aka body acceleration component

Some features of interest for the temperature sensor are:
1. Min and max value
2. Dynamic Range
3. Slope
4. Mean and STD 

Some features of interest for the EDA data are:
1. Range
2. Min and max value
3. Slope
4. Mean and STD
